<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/fourth_new_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fourth Test on KI Checkpoint

After test our sub task implementation with a fixed temperature, this time we visualize the distribution of more probable tokens, tweaking the temperature.

In [4]:
from huggingface_hub import login
login()

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2109, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 2109 (delta 204), reused 201 (delta 113), pack-reused 1794 (from 2)
Receiving objects: 100% (2109/2109), 7.82 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (1347/1347), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 18.12s
Prepared 46 packages in 21.14s
Uninstalled 15 packages in 1.08s
Installed 46 packages in 233ms
 + async-lru==2.1.0
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@46e19ae579f80ce66211afafd1c3c649c569131f)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [6]:
import torch

MODEL_ID = "NONHUMAN-RESEARCH/pi05_ki_training_fruits"
device = "cuda" if torch.cuda.is_available() else "cpu"

PALIGEMMA_EOS_TOKEN = 1
max_decoding_steps = 40
temperature = 0.1

In [7]:
from transformers import AutoTokenizer
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224",
                                          use_fast=False)

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [9]:

from xhuman.policies.pi05.modeling_pi_05 import PI05Policy
# -----------------------------------------------------------------------------
# FIX for AttributeError: 'GemmaRMSNorm' object has no attribute 'weight'
# The custom transformers branch uses adaRMS which removes the weight parameter,
# but the __repr__ method expects it. We monkey-patch it here.
from transformers.models.gemma import modeling_gemma

def fixed_extra_repr(self):
    if hasattr(self, "weight"):
        repr_str = f"{tuple(self.weight.shape)}, eps={self.eps}"
    else:
        repr_str = f"eps={self.eps}"
    if getattr(self, "dense", None) is not None:
        repr_str += f", adaptive=True, cond_dim={self.cond_dim}"
    return repr_str

modeling_gemma.GemmaRMSNorm.extra_repr = fixed_extra_repr
# -----------------------------------------------------------------------------

print("Loading policy...")
policy = PI05Policy.from_pretrained(MODEL_ID)
policy.model.eval()
policy.model.to(device)

Loading policy...


config.json: 0.00B [00:00, ?B/s]

processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Loading model from: NONHUMAN-RESEARCH/pi05_ki_training_fruits


model.safetensors:   0%|          | 0.00/7.47G [00:00<?, ?B/s]

✓ Loaded state dict from model.safetensors
	Missing key(s) in state_dict: "model.paligemma_with_expert.paligemma.model.language_model.embed_tokens.weight". 


PI05Model(
  (paligemma_with_expert): PaliGemmaWithExpertModel(
    (paligemma): PaliGemmaForConditionalGeneration(
      (model): PaliGemmaModel(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(256, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (

In [16]:
import os
import cv2
import torch
import numpy as np

img_share_path = '/content'
img_name_list = ['top.png']
img_list = []

if img_share_path and os.path.exists(img_share_path):
    # Load images from path
    for img_name in img_name_list:
        img_path = os.path.join(img_share_path, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                # Convert BGR to RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_list.append(img)
img_list[0].shape

(768, 1366, 3)

In [17]:
from lerobot.utils.constants import OBS_LANGUAGE_TOKENS, OBS_LANGUAGE_ATTENTION_MASK

batch = {}
if len(img_list) > 0:
    # Prepare image tensor: [H, W, C] -> [1, H, W, C]
    # Normalize to [0, 1] as expected by _preprocess_images (which then scales to [-1, 1])
    img_tensor = torch.from_numpy(img_list[0]).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0) # Add batch dimension -> [1, H, W, C]

    batch["observation.images.top"] = img_tensor

# Process Language Prompt
# Note: PI05 automatically prepends image embeddings, so we DO NOT need an <image> token in the text.
# However, we ensure the prompt ends with a newline for better model performance.
prompt = "Put the fruits in the basket\nSub task: "
if not prompt.endswith("\n"):
    prompt += "\n"

print(f"Tokenizing prompt: '{prompt.strip()}'")
tokenized = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Add to batch (converting mask to bool for compatibility)
batch[OBS_LANGUAGE_TOKENS] = tokenized["input_ids"]
batch[OBS_LANGUAGE_ATTENTION_MASK] = tokenized["attention_mask"].bool()

Tokenizing prompt: 'Put the fruits in the basket
Sub task:'


In [18]:
processed_images, img_masks = policy._preprocess_images(batch)

In [20]:
from xhuman.policies.pi05.modeling_pi_05 import make_att_2d_masks

In [22]:
with torch.no_grad():
    # FIX: Ensure images are in (B, C, H, W) format
    # The error "expected input... to have 3 channels, but got 224" occurs because
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    # Retrieve tokens and masks from the batch dictionary and move them to the correct device
    tokens = batch[OBS_LANGUAGE_TOKENS].to(device)
    masks = batch[OBS_LANGUAGE_ATTENTION_MASK].to(device)

    # -------------------------------------------------------------------------
    # Manual Sampling Loop to inspect Top 5 Tokens
    # -------------------------------------------------------------------------
    import torch.nn.functional as F

    # 1. Embed prefix
    prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
        fixed_processed_images, img_masks, tokens, masks.bool()
    )

    # 2. Prepare masks
    # make_att_2d_masks is in global scope from previous cell execution
    prefix_att_2d_masks = make_att_2d_masks(prefix_pad_masks, prefix_att_masks)
    prefix_position_ids = torch.cumsum(prefix_pad_masks, dim=1) - 1
    prefix_att_2d_masks_4d = policy.model._prepare_attention_masks_4d(prefix_att_2d_masks)

    # 3. Initial Forward Pass
    lang_model = policy.model.paligemma_with_expert.paligemma.language_model
    lang_model.config._attn_implementation = "eager"

    embeddings, past_key_values = policy.model.paligemma_with_expert.forward(
        attention_mask=prefix_att_2d_masks_4d,
        position_ids=prefix_position_ids,
        past_key_values=None,
        inputs_embeds=[prefix_embs, None],
        use_cache=True,
    )

    last_token_embed = embeddings[0][:, -1, :]
    last_logits = policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed)

    batch_size = last_logits.shape[0]
    device = last_logits.device

    prefix_valid_length = torch.sum(prefix_pad_masks, dim=1)
    output_tokens = torch.zeros((batch_size, max_decoding_steps), dtype=torch.long, device=device)
    all_eos = torch.zeros(batch_size, dtype=torch.bool, device=device)

    running_attention_mask = prefix_pad_masks.clone()

    print("Generation Start...")

    # Autoregressive Loop
    for step in range(max_decoding_steps):
        # --- Inspect Probabilities ---
        probs_all = F.softmax(last_logits, dim=-1)
        top_probs, top_indices = torch.topk(probs_all, 20, dim=-1)

        print(f"\nStep {step}:")
        for b in range(batch_size):
            print(f"  Batch {b} Top 5:")
            for k in range(20):
                token_idx = top_indices[b, k].item()
                prob = top_probs[b, k].item()
                token_str = tokenizer.decode([token_idx])
                print(f"    {k+1}: '{token_str}' (p={prob:.4f})")
        # -----------------------------

        # Sample next token
        current_temp = 1.0 # Using 0.7 as in the original cell call
        if current_temp > 0.0:
            probs = F.softmax(last_logits / current_temp, dim=-1)
            token = torch.multinomial(probs, num_samples=1)
        else:
            token = torch.argmax(last_logits, dim=-1, keepdim=True)

        output_tokens[:, step] = token.squeeze(-1)

        # Check for EOS
        all_eos |= (token.squeeze(-1) == PALIGEMMA_EOS_TOKEN)
        if all_eos.all():
            break

        # Feed the new token back in the model
        next_token_embeds = lang_model.embed_tokens(token)
        position_ids = prefix_valid_length[:, None] + step

        new_mask = torch.ones((batch_size, 1), dtype=running_attention_mask.dtype, device=device)
        running_attention_mask = torch.cat([running_attention_mask, new_mask], dim=1)

        embeds_list, past_key_values = policy.model.paligemma_with_expert.forward(
            inputs_embeds=[next_token_embeds, None],
            attention_mask=running_attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=True,
        )

        prefix_output = embeds_list[0]
        last_token_embed = prefix_output[:, -1, :]
        last_logits = policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed)

    print(f"\nOutput tokens shape: {output_tokens.shape}")
    print(f"Output tokens:\n{output_tokens}")

    # Decode the generated tokens
    print("\n" + "="*80)
    # The variable high_level_prompt is not defined. I'm removing it.
    # print("High Level Prompt\n",high_level_prompt)
    print("Decoded output:")
    print("="*80)

    for batch_idx in range(output_tokens.shape[0]):
        batch_tokens = output_tokens[batch_idx]
        # Remove padding (zeros) and decode
        non_zero_tokens = batch_tokens[batch_tokens != 0]
        if len(non_zero_tokens) > 0:
            decoded_text = tokenizer.decode(non_zero_tokens, skip_special_tokens=False)
            print(f"Batch {batch_idx}: {decoded_text}")
        else:
            print(f"Batch {batch_idx}: (empty)")

print("\n" + "="*80)
print("Test completed successfully!")
print("="*80)

Generation Start...

Step 0:
  Batch 0 Top 5:
    1: 'Sub' (p=1.0000)
    2: 'pick' (p=0.0000)
    3: 'adjust' (p=0.0000)
    4: 'put' (p=0.0000)
    5: 'move' (p=0.0000)
    6: 'place' (p=0.0000)
    7: 'return' (p=0.0000)
    8: 'hand' (p=0.0000)
    9: 'pull' (p=0.0000)
    10: 'open' (p=0.0000)
    11: 'push' (p=0.0000)
    12: 'No' (p=0.0000)
    13: 'Yes' (p=0.0000)
    14: 'close' (p=0.0000)
    15: 'hang' (p=0.0000)
    16: 'no' (p=0.0000)
    17: 'go' (p=0.0000)
    18: 'un' (p=0.0000)
    19: 'yes' (p=0.0000)
    20: 'gra' (p=0.0000)

Step 1:
  Batch 0 Top 5:
    1: '<eos>' (p=0.9453)
    2: ' and' (p=0.0016)
    3: ' external' (p=0.0014)
    4: ' contents' (p=0.0009)
    5: ' table' (p=0.0008)
    6: ' the' (p=0.0008)
    7: ' gri' (p=0.0007)
    8: 'put' (p=0.0007)
    9: 'A' (p=0.0007)
    10: 'Yes' (p=0.0006)
    11: ' arm' (p=0.0006)
    12: '.' (p=0.0006)
    13: ' placed' (p=0.0006)
    14: ' place' (p=0.0006)
    15: ' left' (p=0.0005)
    16: ' sports' (p=0.0005)
   